In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import os.path
import errno
import sys




import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch.utils.data import sampler
import torchvision.transforms as T

In [2]:
if sys.version_info[0] == 2:
    import cPickle as pickle
else:
    import pickle

transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
            ])
NUM_TRAIN=49000
train_batch_size=4
test_batch_size=4


cifar10_train = torchvision.datasets.CIFAR10('./cs231n/datasets', train=True, download=True,
                             transform=transform)
loader_train = torch.utils.data.DataLoader(cifar10_train, batch_size=train_batch_size, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = torchvision.datasets.CIFAR10('./cs231n/datasets', train=True, download=True,
                           transform=transform)
loader_val = torch.utils.data.DataLoader(cifar10_val, batch_size=train_batch_size, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = torchvision.datasets.CIFAR10('./cs231n/datasets', train=False, download=True, 
                            transform=transform)
loader_test = torch.utils.data.DataLoader(cifar10_test, batch_size=test_batch_size)





Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [3]:
import torchvision.datasets as dset
from PIL import Image    
import os
import os.path
import errno
import sys
import torch

class CIFAR100(dset.CIFAR10):

    base_folder = '.'
    train_list = [
        ['train', '16019d7e3df5f24257cddd939b257f8d'],
    ]

    test_list = [
        ['test', 'f0ef6b0ae62326f3e7ffdfab6717acfc'],
    ]
    
    def __init__(self, root, train=True,
                 transform=None, target_transform=None,
                 download=False):
        self.root = os.path.expanduser(root)
        self.transform = transform
        self.target_transform = target_transform
        self.train = train  # training set or test set

        if download:
            self.download()

        if not self._check_integrity():
            raise RuntimeError('Dataset not found or corrupted.' +
                               ' You can use download=True to download it')

        # now load the picked numpy arrays
        self.test_data = []
        if self.train:
            self.train_data = []
            self.train_labels = []
            for fentry in self.train_list:
                f = fentry[0]
                file = os.path.join(self.root, self.base_folder, f)
                fo = open(file, 'rb')
                if sys.version_info[0] == 2:
                    entry = pickle.load(fo)
                else:
                    entry = pickle.load(fo, encoding='latin1')
                self.train_data.append(entry['data'])
                if 'labels' in entry:
                    self.train_labels += entry['labels']
                else:
                    self.train_labels += entry['fine_labels']
                fo.close()

            self.train_data = np.concatenate(self.train_data)
            self.train_data = self.train_data.reshape((5000, 3, 32, 32))
            self.train_data = self.train_data.transpose((0, 2, 3, 1))  # convert to HWC
            print(self.train_data.shape)
        else:
            f = 'test'
            file = os.path.join(self.root, self.base_folder, f)
            fo = open(file, 'rb')
            if sys.version_info[0] == 2:
                entry = pickle.load(fo)
            else:
                entry = pickle.load(fo, encoding='latin1')
            self.test_data = np.array(entry['data'])
            if 'labels' in entry:
                self.test_labels = entry['data']
            else:
                self.test_labels = entry['fine_labels']
            fo.close()
            self.test_data = self.test_data.reshape((1000, 3, 32, 32))
            self.test_data = self.test_data.transpose((0, 2, 3, 1))  # convert to HWC
            print(self.test_data.shape)

    def __getitem__(self, index):
        """
        Args:
            index (int): Index

        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        if self.train:
            img, target = self.train_data[index], self.train_labels[index]
        else:
            img, target = self.test_data[index], self.test_labels[index]

        # doing this so that it is consistent with all other datasets
        # to return a PIL Image
        img = Image.fromarray(img.astype('uint8')).convert('RGB')

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target


    def __len__(self):
        if self.train:
            return len(self.train_data)
        else:
            return len(self.test_data)                
            
    def _check_integrity(self):
        return True
    


if sys.version_info[0] == 2:
    import cPickle as pickle
else:
    import pickle

transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
            ])

A_train = CIFAR100('./class1', train=True, download=False,
                             transform=transform)
loaderA_train = torch.utils.data.DataLoader(A_train, batch_size=4)



A_test = CIFAR100('./class1', train=False, download=False,
                             transform=transform)
loaderA_test = torch.utils.data.DataLoader(A_test, batch_size=4)


B_train = CIFAR100('./class2', train=True, download=False,
                             transform=transform)
loaderB_train = torch.utils.data.DataLoader(B_train, batch_size=4)


B_test = CIFAR100('./class2', train=False, download=False,
                             transform=transform)
loaderB_test = torch.utils.data.DataLoader(B_test, batch_size=4)
   


(5000, 32, 32, 3)
(1000, 32, 32, 3)
(5000, 32, 32, 3)
(1000, 32, 32, 3)


In [4]:
class ConvPool_CNN_C(nn.Module):
    def __init__(self, num_classes=10):
        super (ConvPool_CNN_C, self).__init__()
        self.num_classes = num_classes
        
        self.conv1 = nn.Conv2d(3, 96, kernel_size=3, stride=1, padding=1)
        nn.init.kaiming_normal_(self.conv1.weight)
        nn.init.constant_(self.conv1.bias, 0)
        
        self.dropout1 = nn.Dropout2d(0.2)
        
        self.conv2 = nn.Conv2d(96, 96, kernel_size=3, stride=1, padding=1)
        nn.init.kaiming_normal_(self.conv2.weight)
        nn.init.constant_(self.conv2.bias, 0)
        
        self.conv3 = nn.Conv2d(96, 96, kernel_size=3, stride=1, padding=1)
        nn.init.kaiming_normal_(self.conv3.weight)
        nn.init.constant_(self.conv3.bias, 0)
        
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.dropout2 = nn.Dropout(0.5)
        
        self.conv4 = nn.Conv2d(96, 192, kernel_size=3, stride=1, padding=1)
        nn.init.kaiming_normal_(self.conv4.weight)
        nn.init.constant_(self.conv4.bias, 0)
        
        self.conv5 = nn.Conv2d(192, 192, kernel_size=3, stride=1, padding=1)
        nn.init.kaiming_normal_(self.conv5.weight)
        nn.init.constant_(self.conv5.bias, 0)
        
        self.conv6 = nn.Conv2d(192, 192, kernel_size=3, stride=1, padding=1)
        nn.init.kaiming_normal_(self.conv6.weight)
        nn.init.constant_(self.conv6.bias, 0)
        
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.dropout3 = nn.Dropout(0.5)
        
        self.conv7 = nn.Conv2d(192, 192, kernel_size=3, padding=3)
        nn.init.kaiming_normal_(self.conv7.weight)
        nn.init.constant_(self.conv7.bias, 0)
        
        self.conv8 = nn.Conv2d(192, 192, kernel_size=1)
        nn.init.kaiming_normal_(self.conv8.weight)
        nn.init.constant_(self.conv8.bias, 0)
        
        self.conv9 = nn.Conv2d(192, self.num_classes, kernel_size=1)
        nn.init.kaiming_normal_(self.conv9.weight)
        nn.init.constant_(self.conv9.bias, 0)
        
        self.glb_avg = nn.AvgPool2d(6)
        
    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = self.dropout1(out)
        
        out = self.conv2(out)
        out = F.relu(out)
        
        out = self.conv3(out)
        out = F.relu(out)
        
        out = self.maxpool1(out)
        out = self.dropout2(out)
        
        out = self.conv4(out)
        out = F.relu(out)
        
        out = self.conv5(out)
        out = F.relu(out)
        
        out = self.conv6(out)
        out = F.relu(out)
        
        out = self.maxpool2(out)
        out = self.dropout3(out)
        
        out = self.conv7(out)
        out = F.relu(out)
        
        out = self.conv8(out)
        out = F.relu(out)
        
        out = self.conv9(out)
        out = F.relu(out)
        
        out = self.glb_avg(out)
        out = out.view(-1, self.num_classes)
        return out
import new_ALL_Conv
convpool_cnn_c = new_ALL_Conv.ConvPool_CNN_C()

right


In [7]:
from six.moves import cPickle
import os
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch.utils.data import sampler

USE_GPU = True
dtype = torch.float32 # we will be using float throughout this tutorial
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples *100
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples,  acc))
    return acc*100

def running_model_B(run_num, net, net_name, lr_list, epoch_list, loader_train, 
                   loader_test):
    train_batch_size = 4
    test_batch_size = 4
    

    # Constant to control how frequently we print train loss
    print_every = 100

    print('using device:', device)
    
    #net = BaseNet_A()
    net = net.to(device=device)
    criterion = nn.CrossEntropyLoss()        
        
    lr_1, lr_2, lr_3, lr_4 = lr_list[0], lr_list[1], lr_list[2], lr_list[3]
    weight_decay = 0.001

    max_epoch = 350
    display_interval = 500

    train_size = 5000
    test_size = 1000

    num_train_batch = train_size/train_batch_size
    num_test_batch = test_size/test_batch_size

    train_loss = np.zeros((max_epoch,1))
    val_acc = np.zeros((max_epoch,1))
    #train_acc = np.zeros((max_epoch,1))
    #test_loss = np.zeros((max_epoch,1))
    #test_acc = np.zeros((max_epoch,1))

    epoch_acc = [] # max_epoch x num
    print("begin training")
    for epoch in range(max_epoch):
        if(epoch<epoch_list[0]):
            lr = lr_1
        elif(epoch<epoch_list[1]):
            lr = lr_2
        elif(epoch<epoch_list[2]):
            lr = lr_3
        else:
            lr = lr_4
    
        optimizer = optim.SGD( net.parameters(), lr=0.001,
                              momentum=0.9, weight_decay=weight_decay)
    
        running_epoch_loss = 0.
        running_loss_print = 0.
        epoch_total_num = 0
        correct_num = 0
    
        i_acc = []
        #for i, data in enumerate(trainloader):
        for i, data in enumerate(loader_train):
            net.train()
        
            inputs_data, labels_data = data
            inputs, labels = Variable(inputs_data), Variable(labels_data)
            inputs = inputs.to(device=device, dtype=dtype)
            labels = labels.to(device=device, dtype=torch.long)
        
    
        
        
            outputs = net(inputs)
            loss = criterion(outputs, labels)
        
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
            running_epoch_loss += loss.item()
            running_loss_print += loss.item()
            if i%500 == 499: #net a, b, c 500 print once
                
                acc = check_accuracy(loader_test, net)
                i_acc.append(acc)
                print('%d epoch, %5d iteration, loss:%.3f' 
                      %(epoch+1, i+1, running_loss_print/500) )
                running_loss_print = 0.
            
            #_, pred = torch.max(outputs, 1)
            #epoch_total_num += labels.size(0)
            #correct_num += (pred==labels).sum()
        
        
        train_loss[epoch] = running_epoch_loss/num_train_batch
        epoch_acc.append(i_acc)
        
        val_acc[epoch] = np.sum(epoch_acc[epoch])/49
        #val_acc[epoch] = np.sum(epoch_acc[epoch])
        #train_acc[epoch] = correct_num/epoch_total_num*100
    
    
        # test accuracy and loss
        '''
        ts_runningloss_epoch = 0.
        ts_correct = 0
        ts_epoch_num = 0
        for data in testloader:
            inputs_data, labels_data = data
            inputs, labels = Variable(inputs_data), Variable(labels_data)
            inputs = inputs.to(device=device, dtype=dtype)
            labels = labels.to(device=device, dtype=torch.long)
        
            net.eval()
            ts_outputs = net(inputs)
            loss = criterion(outputs, labels)
        
            ts_runningloss_epoch += loss.item()
            _, ts_pred = torch.max(ts_outputs, 1)
        
            ts_epoch_num += labels.size(0)
            ts_correct += (ts_pred==labels).sum()
        test_loss[epoch] = ts_runningloss_epoch/num_test_batch
        test_acc[epoch] = ts_correct/ts_epoch_num*100
        '''
        print(" num %d epoch " %epoch)
        print("####### Training Loss #######")
        print(train_loss[epoch])
        #print("####### Validation Accuracy #######")
        #print(val_acc[epoch])
        #print("####### Training Accuracy #######")
        #print(train_acc[epoch])
        #print("####### Testing Loss #######")
        #print(test_loss[epoch])
        #print("####### Testing Accuracy #######")
        #print(test_acc[epoch])
    
    print('finish training \n')

    



##################################################################################################
    print('now begin saving datum for next step plotting')
    
    save_path = '../datum_for_plotting/run_num_' + str(run_num)+'/'+ net_name
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    f = open(save_path + '/train_loss.save', 'wb')
    cPickle.dump(train_loss, f, protocol=cPickle.HIGHEST_PROTOCOL)
    f.close()

    f = open(save_path + '/val_acc.save', 'wb')
    cPickle.dump(val_acc, f, protocol=cPickle.HIGHEST_PROTOCOL)
    f.close()

    f = open(save_path + '/epoch_acc.save', 'wb')
    cPickle.dump(epoch_acc, f, protocol=cPickle.HIGHEST_PROTOCOL)
    f.close()

    array_epoch_acc = np.array(epoch_acc)
    f = open(save_path + '/array_epoch_acc.save', 'wb')
    cPickle.dump(array_epoch_acc, f, protocol=cPickle.HIGHEST_PROTOCOL)
    f.close()

    f = open(save_path + '/test_acc.save', 'wb')
    cPickle.dump(test_acc, f, protocol=cPickle.HIGHEST_PROTOCOL)
    f.close()



    torch.save(net, save_path+'/'+ net_name +'.pkl') # save whole net structure and params
    torch.save(net.state_dict, save_path+'/'+ net_name +'_params.pkl') # only save model params
 
    
    
    
##################################################################################################    
    print("now plotting accuracies and losses")  
    itern_axis_train = np.array(np.linspace(1,max_epoch,num=max_epoch))
    itern_axis_test = np.array(np.linspace(1,max_epoch, num=max_epoch))

    plt.plot(itern_axis_train, train_loss,'-b.', label='Train')
    plt.xlabel('Epoch')
    plt.ylabel('loss')
    plt.savefig(save_path + '/train_loss' + str(max_epoch) + '.png')
    
    a = np.concatenate(array_epoch_acc)
    length = a.shape[0]
    plt.plot(length, a, '--r', label='Test')
    plt.xlabel('iteration')
    plt.ylabel('loss')
    plt.savefig(save_path + '/testing_accuracy' + str(max_epoch) + '.png')
    
    return net

In [6]:
lr = [0.01, 0.005, 0.001, 0.0005]
epoch = [200, 250, 300] # first20

run_num = 10


#torch.cuda.set_device(3)
#import os

#os.environ["CUDA_VISIBLE_DEVICES"] = "3" ## GPU number= 0, 1, or 2 
net_name1 = 'ConvPool_CNN_C_Class1'
convpool_cnn_c_class1 = running_model_B(run_num, convpool_cnn_c_class1, net_name1, lr, epoch, 
                        loaderA_train, loaderA_test)

using device: cuda
begin training
Checking accuracy on test set
Got 97 / 1000 correct (9.70)
1 epoch,   500 iteration, loss:2.307
Checking accuracy on test set
Got 100 / 1000 correct (10.00)
1 epoch,  1000 iteration, loss:2.302
 num 0 epoch 
####### Training Loss #######
[2.30412699]
Checking accuracy on test set
Got 102 / 1000 correct (10.20)
2 epoch,   500 iteration, loss:2.303
Checking accuracy on test set
Got 104 / 1000 correct (10.40)
2 epoch,  1000 iteration, loss:2.302
 num 1 epoch 
####### Training Loss #######
[2.30241282]
Checking accuracy on test set
Got 98 / 1000 correct (9.80)
3 epoch,   500 iteration, loss:2.302
Checking accuracy on test set
Got 114 / 1000 correct (11.40)
3 epoch,  1000 iteration, loss:2.301
 num 2 epoch 
####### Training Loss #######
[2.30114633]
Checking accuracy on test set
Got 158 / 1000 correct (15.80)
4 epoch,   500 iteration, loss:2.301
Checking accuracy on test set
Got 168 / 1000 correct (16.80)
4 epoch,  1000 iteration, loss:2.293
 num 3 epoch 
#

Got 705 / 1000 correct (70.50)
33 epoch,   500 iteration, loss:0.586
Checking accuracy on test set
Got 692 / 1000 correct (69.20)
33 epoch,  1000 iteration, loss:0.589
 num 32 epoch 
####### Training Loss #######
[0.57235052]
Checking accuracy on test set
Got 725 / 1000 correct (72.50)
34 epoch,   500 iteration, loss:0.546
Checking accuracy on test set
Got 693 / 1000 correct (69.30)
34 epoch,  1000 iteration, loss:0.581
 num 33 epoch 
####### Training Loss #######
[0.55203993]
Checking accuracy on test set
Got 717 / 1000 correct (71.70)
35 epoch,   500 iteration, loss:0.523
Checking accuracy on test set
Got 711 / 1000 correct (71.10)
35 epoch,  1000 iteration, loss:0.528
 num 34 epoch 
####### Training Loss #######
[0.51936967]
Checking accuracy on test set
Got 694 / 1000 correct (69.40)
36 epoch,   500 iteration, loss:0.508
Checking accuracy on test set
Got 681 / 1000 correct (68.10)
36 epoch,  1000 iteration, loss:0.527
 num 35 epoch 
####### Training Loss #######
[0.51074522]
Checki

Checking accuracy on test set
Got 741 / 1000 correct (74.10)
65 epoch,  1000 iteration, loss:0.191
 num 64 epoch 
####### Training Loss #######
[0.18886367]
Checking accuracy on test set
Got 762 / 1000 correct (76.20)
66 epoch,   500 iteration, loss:0.166
Checking accuracy on test set
Got 749 / 1000 correct (74.90)
66 epoch,  1000 iteration, loss:0.186
 num 65 epoch 
####### Training Loss #######
[0.17659084]
Checking accuracy on test set
Got 750 / 1000 correct (75.00)
67 epoch,   500 iteration, loss:0.159
Checking accuracy on test set
Got 764 / 1000 correct (76.40)
67 epoch,  1000 iteration, loss:0.197
 num 66 epoch 
####### Training Loss #######
[0.18133748]
Checking accuracy on test set
Got 751 / 1000 correct (75.10)
68 epoch,   500 iteration, loss:0.153
Checking accuracy on test set
Got 749 / 1000 correct (74.90)
68 epoch,  1000 iteration, loss:0.190
 num 67 epoch 
####### Training Loss #######
[0.17054572]
Checking accuracy on test set
Got 742 / 1000 correct (74.20)
69 epoch,   50

Got 747 / 1000 correct (74.70)
97 epoch,  1000 iteration, loss:0.117
 num 96 epoch 
####### Training Loss #######
[0.12809811]
Checking accuracy on test set
Got 768 / 1000 correct (76.80)
98 epoch,   500 iteration, loss:0.100
Checking accuracy on test set
Got 750 / 1000 correct (75.00)
98 epoch,  1000 iteration, loss:0.101
 num 97 epoch 
####### Training Loss #######
[0.09612415]
Checking accuracy on test set
Got 760 / 1000 correct (76.00)
99 epoch,   500 iteration, loss:0.111
Checking accuracy on test set
Got 717 / 1000 correct (71.70)
99 epoch,  1000 iteration, loss:0.091
 num 98 epoch 
####### Training Loss #######
[0.09893966]
Checking accuracy on test set
Got 749 / 1000 correct (74.90)
100 epoch,   500 iteration, loss:0.100
Checking accuracy on test set
Got 737 / 1000 correct (73.70)
100 epoch,  1000 iteration, loss:0.152
 num 99 epoch 
####### Training Loss #######
[0.1286833]
Checking accuracy on test set
Got 742 / 1000 correct (74.20)
101 epoch,   500 iteration, loss:0.087
Chec

Checking accuracy on test set
Got 751 / 1000 correct (75.10)
129 epoch,  1000 iteration, loss:0.111
 num 128 epoch 
####### Training Loss #######
[0.09583357]
Checking accuracy on test set
Got 752 / 1000 correct (75.20)
130 epoch,   500 iteration, loss:0.048
Checking accuracy on test set
Got 759 / 1000 correct (75.90)
130 epoch,  1000 iteration, loss:0.092
 num 129 epoch 
####### Training Loss #######
[0.06993333]
Checking accuracy on test set
Got 772 / 1000 correct (77.20)
131 epoch,   500 iteration, loss:0.083
Checking accuracy on test set
Got 749 / 1000 correct (74.90)
131 epoch,  1000 iteration, loss:0.103
 num 130 epoch 
####### Training Loss #######
[0.09804816]
Checking accuracy on test set
Got 761 / 1000 correct (76.10)
132 epoch,   500 iteration, loss:0.067
Checking accuracy on test set
Got 745 / 1000 correct (74.50)
132 epoch,  1000 iteration, loss:0.075
 num 131 epoch 
####### Training Loss #######
[0.08101595]
Checking accuracy on test set
Got 740 / 1000 correct (74.00)
133

...

Got 760 / 1000 correct (76.00)
224 epoch,  1000 iteration, loss:0.082
 num 223 epoch 
####### Training Loss #######
[0.06168237]
Checking accuracy on test set
Got 747 / 1000 correct (74.70)
225 epoch,   500 iteration, loss:0.062
Checking accuracy on test set
Got 748 / 1000 correct (74.80)
225 epoch,  1000 iteration, loss:0.065
 num 224 epoch 
####### Training Loss #######
[0.06002546]
Checking accuracy on test set
Got 747 / 1000 correct (74.70)
226 epoch,   500 iteration, loss:0.054
Checking accuracy on test set
Got 708 / 1000 correct (70.80)
226 epoch,  1000 iteration, loss:0.079
 num 225 epoch 
####### Training Loss #######
[0.07084378]
Checking accuracy on test set
Got 765 / 1000 correct (76.50)
227 epoch,   500 iteration, loss:0.060
Checking accuracy on test set
Got 741 / 1000 correct (74.10)
227 epoch,  1000 iteration, loss:0.074
 num 226 epoch 
####### Training Loss #######
[0.07375768]
Checking accuracy on test set
Got 776 / 1000 correct (77.60)
228 epoch,   500 iteration, loss:

Checking accuracy on test set
Got 743 / 1000 correct (74.30)
256 epoch,  1000 iteration, loss:0.048
 num 255 epoch 
####### Training Loss #######
[0.05973746]
Checking accuracy on test set
Got 758 / 1000 correct (75.80)
257 epoch,   500 iteration, loss:0.059
Checking accuracy on test set
Got 755 / 1000 correct (75.50)
257 epoch,  1000 iteration, loss:0.055
 num 256 epoch 
####### Training Loss #######
[0.06335831]
Checking accuracy on test set
Got 770 / 1000 correct (77.00)
258 epoch,   500 iteration, loss:0.078
Checking accuracy on test set
Got 760 / 1000 correct (76.00)
258 epoch,  1000 iteration, loss:0.056
 num 257 epoch 
####### Training Loss #######
[0.06602053]
Checking accuracy on test set
Got 764 / 1000 correct (76.40)
259 epoch,   500 iteration, loss:0.062
Checking accuracy on test set
Got 764 / 1000 correct (76.40)
259 epoch,  1000 iteration, loss:0.032
 num 258 epoch 
####### Training Loss #######
[0.05428074]
Checking accuracy on test set
Got 771 / 1000 correct (77.10)
260

Got 761 / 1000 correct (76.10)
288 epoch,   500 iteration, loss:0.056
Checking accuracy on test set
Got 755 / 1000 correct (75.50)
288 epoch,  1000 iteration, loss:0.071
 num 287 epoch 
####### Training Loss #######
[0.06381788]
Checking accuracy on test set
Got 766 / 1000 correct (76.60)
289 epoch,   500 iteration, loss:0.040
Checking accuracy on test set
Got 749 / 1000 correct (74.90)
289 epoch,  1000 iteration, loss:0.046
 num 288 epoch 
####### Training Loss #######
[0.05122744]
Checking accuracy on test set
Got 768 / 1000 correct (76.80)
290 epoch,   500 iteration, loss:0.040
Checking accuracy on test set
Got 759 / 1000 correct (75.90)
290 epoch,  1000 iteration, loss:0.062
 num 289 epoch 
####### Training Loss #######
[0.05901718]
Checking accuracy on test set
Got 744 / 1000 correct (74.40)
291 epoch,   500 iteration, loss:0.040
Checking accuracy on test set
Got 770 / 1000 correct (77.00)
291 epoch,  1000 iteration, loss:0.055
 num 290 epoch 
####### Training Loss #######
[0.0518

 num 318 epoch 
####### Training Loss #######
[0.06326061]
Checking accuracy on test set
Got 771 / 1000 correct (77.10)
320 epoch,   500 iteration, loss:0.068
Checking accuracy on test set
Got 779 / 1000 correct (77.90)
320 epoch,  1000 iteration, loss:0.044
 num 319 epoch 
####### Training Loss #######
[0.05212913]
Checking accuracy on test set
Got 749 / 1000 correct (74.90)
321 epoch,   500 iteration, loss:0.048
Checking accuracy on test set
Got 717 / 1000 correct (71.70)
321 epoch,  1000 iteration, loss:0.072
 num 320 epoch 
####### Training Loss #######
[0.06214204]
Checking accuracy on test set
Got 761 / 1000 correct (76.10)
322 epoch,   500 iteration, loss:0.056
Checking accuracy on test set
Got 761 / 1000 correct (76.10)
322 epoch,  1000 iteration, loss:0.054
 num 321 epoch 
####### Training Loss #######
[0.05311508]
Checking accuracy on test set
Got 763 / 1000 correct (76.30)
323 epoch,   500 iteration, loss:0.032
Checking accuracy on test set
Got 766 / 1000 correct (76.60)
323

NameError: name 'cPickle' is not defined

In [8]:
convpool_cnn_c_class2 = new_ALL_Conv.ConvPool_CNN_C()

In [10]:
net_name2 = 'ConvPool_CNN_C_Class2'
convpool_cnn_c_class2 = running_model_B(run_num, convpool_cnn_c_class2, net_name2, 
                        lr, epoch, loaderB_train, loaderB_test)

using device: cuda
begin training
Checking accuracy on test set
Got 314 / 1000 correct (31.40)
1 epoch,   500 iteration, loss:2.103
Checking accuracy on test set
Got 385 / 1000 correct (38.50)
1 epoch,  1000 iteration, loss:1.851
 num 0 epoch 
####### Training Loss #######
[1.92732715]
Checking accuracy on test set
Got 397 / 1000 correct (39.70)
2 epoch,   500 iteration, loss:1.673
Checking accuracy on test set
Got 495 / 1000 correct (49.50)
2 epoch,  1000 iteration, loss:1.579
 num 1 epoch 
####### Training Loss #######
[1.59569475]
Checking accuracy on test set
Got 532 / 1000 correct (53.20)
3 epoch,   500 iteration, loss:1.481
Checking accuracy on test set
Got 535 / 1000 correct (53.50)
3 epoch,  1000 iteration, loss:1.401
 num 2 epoch 
####### Training Loss #######
[1.41575386]
Checking accuracy on test set
Got 555 / 1000 correct (55.50)
4 epoch,   500 iteration, loss:1.328
Checking accuracy on test set
Got 583 / 1000 correct (58.30)
4 epoch,  1000 iteration, loss:1.303
 num 3 epoc

Checking accuracy on test set
Got 703 / 1000 correct (70.30)
33 epoch,   500 iteration, loss:0.398
Checking accuracy on test set
Got 699 / 1000 correct (69.90)
33 epoch,  1000 iteration, loss:0.406
 num 32 epoch 
####### Training Loss #######
[0.39166199]
Checking accuracy on test set
Got 714 / 1000 correct (71.40)
34 epoch,   500 iteration, loss:0.389
Checking accuracy on test set
Got 724 / 1000 correct (72.40)
34 epoch,  1000 iteration, loss:0.371
 num 33 epoch 
####### Training Loss #######
[0.37838244]
Checking accuracy on test set
Got 722 / 1000 correct (72.20)
35 epoch,   500 iteration, loss:0.390
Checking accuracy on test set
Got 711 / 1000 correct (71.10)
35 epoch,  1000 iteration, loss:0.358
 num 34 epoch 
####### Training Loss #######
[0.36380899]
Checking accuracy on test set
Got 722 / 1000 correct (72.20)
36 epoch,   500 iteration, loss:0.371
Checking accuracy on test set
Got 728 / 1000 correct (72.80)
36 epoch,  1000 iteration, loss:0.359
 num 35 epoch 
####### Training Lo

Got 718 / 1000 correct (71.80)
65 epoch,   500 iteration, loss:0.152
Checking accuracy on test set
Got 734 / 1000 correct (73.40)
65 epoch,  1000 iteration, loss:0.157
 num 64 epoch 
####### Training Loss #######
[0.16708147]
Checking accuracy on test set
Got 740 / 1000 correct (74.00)
66 epoch,   500 iteration, loss:0.137
Checking accuracy on test set
Got 738 / 1000 correct (73.80)
66 epoch,  1000 iteration, loss:0.139
 num 65 epoch 
####### Training Loss #######
[0.14168075]
Checking accuracy on test set
Got 727 / 1000 correct (72.70)
67 epoch,   500 iteration, loss:0.130
Checking accuracy on test set
Got 734 / 1000 correct (73.40)
67 epoch,  1000 iteration, loss:0.151
 num 66 epoch 
####### Training Loss #######
[0.15271061]
Checking accuracy on test set
Got 703 / 1000 correct (70.30)
68 epoch,   500 iteration, loss:0.153
Checking accuracy on test set
Got 733 / 1000 correct (73.30)
68 epoch,  1000 iteration, loss:0.153
 num 67 epoch 
####### Training Loss #######
[0.16211497]
Checki

Checking accuracy on test set
Got 740 / 1000 correct (74.00)
97 epoch,  1000 iteration, loss:0.116
 num 96 epoch 
####### Training Loss #######
[0.1049616]
Checking accuracy on test set
Got 710 / 1000 correct (71.00)
98 epoch,   500 iteration, loss:0.093
Checking accuracy on test set
Got 751 / 1000 correct (75.10)
98 epoch,  1000 iteration, loss:0.085
 num 97 epoch 
####### Training Loss #######
[0.09674953]
Checking accuracy on test set
Got 725 / 1000 correct (72.50)
99 epoch,   500 iteration, loss:0.119
Checking accuracy on test set
Got 735 / 1000 correct (73.50)
99 epoch,  1000 iteration, loss:0.116
 num 98 epoch 
####### Training Loss #######
[0.11503342]
Checking accuracy on test set
Got 728 / 1000 correct (72.80)
100 epoch,   500 iteration, loss:0.132
Checking accuracy on test set
Got 746 / 1000 correct (74.60)
100 epoch,  1000 iteration, loss:0.079
 num 99 epoch 
####### Training Loss #######
[0.10011977]
Checking accuracy on test set
Got 725 / 1000 correct (72.50)
101 epoch,   

Got 745 / 1000 correct (74.50)
129 epoch,   500 iteration, loss:0.090
Checking accuracy on test set
Got 721 / 1000 correct (72.10)
129 epoch,  1000 iteration, loss:0.072
 num 128 epoch 
####### Training Loss #######
[0.09556474]
Checking accuracy on test set
Got 735 / 1000 correct (73.50)
130 epoch,   500 iteration, loss:0.068
Checking accuracy on test set
Got 748 / 1000 correct (74.80)
130 epoch,  1000 iteration, loss:0.070
 num 129 epoch 
####### Training Loss #######
[0.07683299]
Checking accuracy on test set
Got 745 / 1000 correct (74.50)
131 epoch,   500 iteration, loss:0.070
Checking accuracy on test set
Got 750 / 1000 correct (75.00)
131 epoch,  1000 iteration, loss:0.076
 num 130 epoch 
####### Training Loss #######
[0.07436359]
Checking accuracy on test set
Got 731 / 1000 correct (73.10)
132 epoch,   500 iteration, loss:0.071
Checking accuracy on test set
Got 711 / 1000 correct (71.10)
132 epoch,  1000 iteration, loss:0.071
 num 131 epoch 
####### Training Loss #######
[0.0804

...

Checking accuracy on test set
Got 733 / 1000 correct (73.30)
224 epoch,  1000 iteration, loss:0.041
 num 223 epoch 
####### Training Loss #######
[0.05096554]
Checking accuracy on test set
Got 737 / 1000 correct (73.70)
225 epoch,   500 iteration, loss:0.050
Checking accuracy on test set
Got 720 / 1000 correct (72.00)
225 epoch,  1000 iteration, loss:0.072
 num 224 epoch 
####### Training Loss #######
[0.06399252]
Checking accuracy on test set
Got 747 / 1000 correct (74.70)
226 epoch,   500 iteration, loss:0.060
Checking accuracy on test set
Got 751 / 1000 correct (75.10)
226 epoch,  1000 iteration, loss:0.094
 num 225 epoch 
####### Training Loss #######
[0.07325612]
Checking accuracy on test set
Got 700 / 1000 correct (70.00)
227 epoch,   500 iteration, loss:0.077
Checking accuracy on test set
Got 726 / 1000 correct (72.60)
227 epoch,  1000 iteration, loss:0.089
 num 226 epoch 
####### Training Loss #######
[0.07127344]
Checking accuracy on test set
Got 739 / 1000 correct (73.90)
228

Got 753 / 1000 correct (75.30)
256 epoch,   500 iteration, loss:0.039
Checking accuracy on test set
Got 744 / 1000 correct (74.40)
256 epoch,  1000 iteration, loss:0.042
 num 255 epoch 
####### Training Loss #######
[0.04809539]
Checking accuracy on test set
Got 745 / 1000 correct (74.50)
257 epoch,   500 iteration, loss:0.086
Checking accuracy on test set
Got 747 / 1000 correct (74.70)
257 epoch,  1000 iteration, loss:0.038
 num 256 epoch 
####### Training Loss #######
[0.05872137]
Checking accuracy on test set
Got 719 / 1000 correct (71.90)
258 epoch,   500 iteration, loss:0.084
Checking accuracy on test set
Got 745 / 1000 correct (74.50)
258 epoch,  1000 iteration, loss:0.060
 num 257 epoch 
####### Training Loss #######
[0.07574531]
Checking accuracy on test set
Got 715 / 1000 correct (71.50)
259 epoch,   500 iteration, loss:0.078
Checking accuracy on test set
Got 746 / 1000 correct (74.60)
259 epoch,  1000 iteration, loss:0.057
 num 258 epoch 
####### Training Loss #######
[0.0643

 num 286 epoch 
####### Training Loss #######
[0.04797518]
Checking accuracy on test set
Got 751 / 1000 correct (75.10)
288 epoch,   500 iteration, loss:0.065
Checking accuracy on test set
Got 740 / 1000 correct (74.00)
288 epoch,  1000 iteration, loss:0.067
 num 287 epoch 
####### Training Loss #######
[0.05876497]
Checking accuracy on test set
Got 735 / 1000 correct (73.50)
289 epoch,   500 iteration, loss:0.059
Checking accuracy on test set
Got 751 / 1000 correct (75.10)
289 epoch,  1000 iteration, loss:0.082
 num 288 epoch 
####### Training Loss #######
[0.06843176]
Checking accuracy on test set
Got 750 / 1000 correct (75.00)
290 epoch,   500 iteration, loss:0.049
Checking accuracy on test set
Got 723 / 1000 correct (72.30)
290 epoch,  1000 iteration, loss:0.041
 num 289 epoch 
####### Training Loss #######
[0.05907758]
Checking accuracy on test set
Got 733 / 1000 correct (73.30)
291 epoch,   500 iteration, loss:0.034
Checking accuracy on test set
Got 743 / 1000 correct (74.30)
291

Got 743 / 1000 correct (74.30)
319 epoch,  1000 iteration, loss:0.044
 num 318 epoch 
####### Training Loss #######
[0.06770256]
Checking accuracy on test set
Got 749 / 1000 correct (74.90)
320 epoch,   500 iteration, loss:0.057
Checking accuracy on test set
Got 744 / 1000 correct (74.40)
320 epoch,  1000 iteration, loss:0.052
 num 319 epoch 
####### Training Loss #######
[0.05856353]
Checking accuracy on test set
Got 743 / 1000 correct (74.30)
321 epoch,   500 iteration, loss:0.039
Checking accuracy on test set
Got 737 / 1000 correct (73.70)
321 epoch,  1000 iteration, loss:0.084
 num 320 epoch 
####### Training Loss #######
[0.06458165]
Checking accuracy on test set
Got 756 / 1000 correct (75.60)
322 epoch,   500 iteration, loss:0.037
Checking accuracy on test set
Got 744 / 1000 correct (74.40)
322 epoch,  1000 iteration, loss:0.041
 num 321 epoch 
####### Training Loss #######
[0.041776]
Checking accuracy on test set
Got 687 / 1000 correct (68.70)
323 epoch,   500 iteration, loss:0.

NameError: name 'test_acc' is not defined